### Hi guys! This notebook will slowly enter into competition and achieve high scores.

#### Please support and stay updated.

### This notebook holds analysis and visualization for Players, Games, Plays, PffScouting data. To view analysis and visualizations of year wise tracking data check this notebook -> [NFL yearly tracking data complete !! 🏈](https://www.kaggle.com/zwartfreak/nfl-yearly-tracking-data-complete)

##### P.S. This notebook is not completed yet, building it daily.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

players = pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
games = pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
plays = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
pffscouting = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
#tracking2018 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2018.csv')
#tracking2019 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2019.csv')
#tracking2020 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')

> ## Data analysis

In [ ]:
players.shape, games.shape, plays.shape

#### There are different datasets so we will analyse them one by one.

### 1. Players

In [ ]:
players.head()

In [ ]:
players.isnull().sum()

##### Let's drop the player's data whose 'birthDate' is not avalable.
##### Also, let's drop the missing data from 'collegeName'.

In [ ]:
players.dropna(subset=['collegeName', 'birthDate'], inplace=True, how='any')
players = players.reset_index(drop=True)
players.isnull().sum(), players.shape

##### So players dataset is free of null values

#### Let's convert 'height' to a simple integer and 'birthDate' to 'Age'

In [ ]:
for i in range(0,(len(players.height)-1)):
    l = len(players.height[i])
    if '-' in players.height[i]:
        if l ==3:
            players.height[i] = int(players.height[i][0])*12 + int(players.height[i][l-1])
        else:
            players.height[i] = int(players.height[i][0])*12 + int(players.height[i][l-1]) + 10
    else:
        players.height[i] = int(players.height[i])  
        
#players.height = players.height.astype('int')

In [ ]:
players.height.unique()

##### There is still a value which is not getting converted in to simple integer.

In [ ]:
from datetime import datetime, date
today = date.today()

players.birthDate = pd.to_datetime(players.birthDate)
players.birthDate = players.birthDate.dt.strftime("%Y/%m/%d")

for j in range(0,(len(players.height)-1)):
    born = datetime.strptime(str(players.birthDate[j]), "%Y/%m/%d").date()
    players.birthDate[j] = today.year - born.year - ((today.month, today.day) < (born.month, born.day))

players['age'] = players.birthDate
players.drop(['birthDate'], axis=1, inplace=True)

#players.birthDate = players.birthDate.astype('int')

In [ ]:
players.age.unique()

##### There is still a value which is not getting converted in to simple age just like above.

In [ ]:
players.head()

> ## Wow that was quite a task, relaxed to do it.

In [ ]:
players.dtypes

##### Some are int and other are object, we will treat them later as required

### 2. Games

In [ ]:
games.head()

In [ ]:
games.isnull().sum()

##### No null values in Games

### 3. Plays

In [ ]:
plays.head()

In [ ]:
plays.isnull().sum().sum()

##### That's hell lot of NULL values

In [ ]:
plays.isnull().sum()

##### As most of the values are null in 'kickBlockerId', 'penaltyCodes', 'penaltyJerseyNumbers', 'penaltyYards', 'passResult', 'returnerId'

In [ ]:
plays.drop(columns=['kickBlockerId', 'penaltyCodes', 'penaltyJerseyNumbers', 'penaltyYards', 'passResult', 'returnerId', 'yardlineSide'], inplace=True)

In [ ]:
plays.kickReturnYardage.fillna(round(plays.kickReturnYardage.mean()), inplace=True)
plays.kickLength.fillna(round(plays.kickLength.mean()), inplace=True)
plays.isnull().sum().sum(), round(plays.kickReturnYardage.mean()), round(plays.kickLength.mean())

plays.dropna(subset=['kickerId'],inplace=True)

In [ ]:
plays.isnull().sum().sum()

##### So plays dataset is free of null values

In [ ]:
players.columns, games.columns, plays.columns

##### We can join games and plays dataset as gameId is the common column between them.

In [ ]:
games_plays = pd.merge(games, plays, on='gameId')
games_plays.dtypes

##### Different types of columns are there, we will treat them later as required

In [ ]:
games_plays.columns

In [ ]:
players.columns

> ## Data visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

##### We can answer so many questions using Visualization.

#### Q1. Which 'season' has most number of games?

In [ ]:
games_plays.season.value_counts().plot(kind='barh', figsize=(10,6), color='yellow')
plt.show()

#You can just do it with games_plays.season.value_counts() this also but visualization feels better.

#### Q2. Find young and tall players.
As we have converted above, we can use age and height here.

In [ ]:
plt.subplot(1,2,1)
players.height.value_counts().plot(kind='bar', figsize=(14,6), color='green')
plt.xlabel('height')

plt.subplot(1,2,2)
players.age.value_counts().plot(kind='bar', figsize=(14,6), color='orange')
plt.xlabel('age')

plt.plot()

#### Q3. Which college has produced maximum number of players?

In [ ]:
players.collegeName.nunique()

##### As too many values unique values are there, let's not plot it.

In [ ]:
players.collegeName.value_counts()

#### Q4. What is the relation between 'kicklength' and 'yardlinenumber'?